## Alg Evolutivo

- Representação: Definir como as soluções candidatas (indivíduos) são representadas (por exemplo, vetores de variáveis de decisão).
- Inicialização: Gerar uma população inicial de soluções de maneira aleatória ou heurística.
- Seleção: Escolher os indivíduos que participarão do processo de reprodução com base em sua aptidão. que determina o quão "fit" ou adequado um indivíduo é para o ambiente ou para a tarefa que está sendo otimizada.
- Cruzamento: AA combinação de dois indivíduos para criar descendentes que compartilham características de ambos os pais. Isto introduz diversidade genética na população. Isto introduz diversidade genética na população.
-Mutação: Uma mudança aleatória em um indivíduo, que pode ajudar a população a explorar novas partes do espaço de solução e evitar o problema de convergência prematura para um ótimo local.
- Gerações: O algoritmo evolui através de várias gerações, onde em cada geração os passos de seleção, cruzamento e mutação são repetidos.

- Convergência: O algoritmo continua até que uma condição de parada seja satisfeita, que pode ser um número fixo de gerações, uma aptidão desejada que foi alcançada, ou falta de progresso significativo.

- Avaliação: Calcular a aptidão de cada solução com base em um modelo de programação linear.
- Substituição: Decidir quais soluções serão mantidas para a próxima geração.

## REAC + Conjutno Elite

### Amostras de individuo

In [120]:
import random
from faker import Faker
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

fake = Faker("pt_BR")


class Individual:
    def __init__(self, nome, idade, sexo,peso,altura):
        self.nome = nome
        self.idade = idade
        self.sexo = sexo
        self.peso = peso
        self.altura = altura
        self.QI = random.randint(20, 130)

        self.imc = self.calculate_imc()
        self.fitness = self.calculate_fitness()

        

    def calculate_imc(self):
        return round(self.peso/(self.altura**2),2)

    # Example fitness function
    def calculate_fitness(self):
        # Define os pesos para cada critério
        peso_imc = 0.4  # 40% do fitness vem do IMC
        peso_idade = 0.2  # 20% do fitness vem da idade
        peso_qi = 0.4  # 40% do fitness vem do QI

        # Calcula o score do IMC (considera-se o IMC normal como mais saudável)
        imc_score = (24.9 - abs(self.calculate_imc() - 24.9)) / 24.9

        # Calcula o score da idade (considera-se 50 anos como pico de 'saúde')
        idade_score = max(0, 1 - abs(self.idade - 50) / 50)

        # Calcula o score do QI (considera-se 130 como QI máximo para pontuação máxima)
        qi_score = self.QI / 130

        # Calcula o fitness geral
        fitness_total = (imc_score * peso_imc +
                         idade_score * peso_idade +
                         qi_score * peso_qi)

        # Normaliza o fitness_total para estar no intervalo de 0 a 100
        fitness_normalizado = round(max(0, min(100, fitness_total * 100)),1)

        return fitness_normalizado

In [121]:
import pandas as pd
import random
import networkx as nx
import matplotlib.pyplot as plt

def calculate_imc(peso,altura):
        return round(peso/(altura**2),2)

def plot_family_graph_inverted_triangle(male_individual, female_individual, herdeiro):
    G = nx.DiGraph()  # Usa DiGraph para direcionar as arestas se necessário
    
    # Adiciona os nós
    G.add_node(male_individual['nome'], sexo='M', imc=male_individual['IMC'])
    G.add_node(female_individual['nome'], sexo='F', imc = female_individual['IMC'])
    G.add_node(herdeiro[0]['nome'], sexo=herdeiro[0]['sexo'], imc = herdeiro[0]['IMC'])
    
    # Adiciona as arestas
    G.add_edge(male_individual['nome'], herdeiro[0]['nome'])
    G.add_edge(female_individual['nome'], herdeiro[0]['nome'])
    
    # Define as posições manualmente
    pos = {
        male_individual['nome']: (0, 0),  # Pai à esquerda
        female_individual['nome']: (2, 0),  # Mãe à direita
        herdeiro[0]['nome']: (1, -2)  # Herdeiro abaixo, formando um triângulo
    }
    
    # Desenha o grafo
    labels = {node: f"{node}\n{G.nodes[node]['sexo']} - {G.nodes[node]['imc']}" for node in G.nodes}

    nx.draw(G, pos, with_labels=False, node_color='skyblue', node_size=3000, edge_color='gray')
    #nx.draw(G, pos, with_labels=True, node_color="skyblue", edge_color="gray", arrows=False)
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=8)

    plt.title("Grafo Família - Triângulo Invertido")
    plt.show()

    
# Função para selecionar um par e verificar se já foram emparelhados
def select_pair(df_male, df_female, paired_names):
    while True:
        male = df_male.sample(1).iloc[0]
        female = df_female.sample(1).iloc[0]
        
        # Cria uma tupla com os nomes, independente da ordem
        pair = tuple(sorted([male['nome'], female['nome']]))
        
        # Verifica se o par já existe
        if pair not in paired_names:
            paired_names.add(pair)
            return male, female
        
# Suponha que df seja o seu DataFrame
# Separe os DataFrames por sexo
def cruzamentoElite(df):
    df_male = df[df['sexo'] == 'M']
    df_female = df[df['sexo'] == 'F']

    # Lista para armazenar os pares que já foram feitos
    paired_names = set()

    # Seleciona um par
    male_individual, female_individual = select_pair(df_male, df_female, paired_names)

    # Agora você tem dois indivíduos para fazer o "cruzamento"
    # Você pode criar um novo indivíduo (herdeiro) baseado nas características deles
    # Por exemplo:

    
    herdeiro = [{
        'nome': fake.unique.first_name_male() + " " +  fake.unique.last_name_male(),
        'idade': fake.random_int(min=18, max=80),
        'sexo': random.choice(['M', 'F']),
        'peso[KG]': (male_individual['peso[KG]'] + female_individual['peso[KG]']) / 2,
        'altura': (male_individual['altura'] + female_individual['altura']) / 2,
        "IMC":calculate_imc(male_individual['peso[KG]'],female_individual["altura"])
    
    }]


    # Supondo que male_individual, female_individual e herdeiro já foram definidos
    plot_family_graph_inverted_triangle(male_individual, female_individual, herdeiro)


### População

In [122]:
class Population:
    def __init__(self, size):
        self.conjunto_elite = []
        self.used_names = set()
        self.individuals = [self.create_individual() for _ in range(size)]

    def create_individual(self):
        while True:
            sexo = random.choice(["M", "F"])
            if sexo == 'M':
                nome = fake.unique.first_name_male() + " " + fake.unique.last_name_male()
            else:
                nome = fake.unique.first_name_female() + " " + fake.unique.last_name_female()
            if nome not in self.used_names:
                self.used_names.add(nome)
                break
        
        idade = fake.random_int(min=18, max=80)
        peso = fake.random_int(min=45, max=150)
        altura = round(random.uniform(1.20,2.10),2)
        
        return Individual(nome, idade, sexo, peso, altura)
    
    def get_elite_set(self):
        elite_set = []
        for ind in self.individuals:
            if (18.5 < ind.imc <= 25) and (ind.altura >= 1.75) and (ind.QI>=120):
                elite_set.append(ind)
        return elite_set

    def getConjuntoElite(self):
        for ind in self.individuals:
            if (18.5 < ind.imc <= 25) and (ind.altura >= 1.75):
                self.conjunto_elite.append(ind)
                return ind



    def evaluate(self):
        for individual in self.individuals:
            individual.fitness = individual.calculate_fitness()
            individual.imc = individual.calculate_imc()

            #Critério de IMC
            if((individual.imc >= 18.5) and (individual.imc <= 25)):
                #!print(f"Nome: {individual.nome}, imc = {individual.imc}, idade = {individual.idade}, data: {individual.peso}KG x {individual.altura}m ")
                self.conjunto_elite.append(individual)

    def selection(self):
        """ Esta função é responsável por selecionar os indivíduos mais aptos da população para reprodução. Ao classificar os indivíduos com base em seu desempenho (fitness) e escolher os melhores, a seleção direciona o algoritmo para soluções de maior qualidade. A ideia é simular o processo natural em que os indivíduos mais adaptados ao ambiente têm maiores chances de sobreviver e se reproduzir."""
        # Simple selection: sort by fitness and take the top half
        self.individuals.sort(key=lambda x: x.fitness, reverse=True)
        self.individuals = self.individuals[:len(self.individuals)//2]

    def crossover(self):
        """A função de cruzamento imita a reprodução sexual na natureza, onde dois indivíduos (pais) combinam partes de seus genomas para criar descendentes. Essa mistura de material genético pode produzir novos indivíduos com características únicas, potencialmente mais aptos que seus pais. Isso introduz diversidade genética na população, o que é crucial para explorar o espaço de solução em busca de ótimos globais."""
        new_generation = []
        for _ in range(len(self.individuals)//2):
            parent1 = random.choice(self.individuals)
            parent2 = random.choice(self.individuals)
            child1 = Individual(parent1.nome, (parent1.idade + parent2.idade) // 2, parent1.sexo, parent1.peso,parent1.altura)
            child2 = Individual(parent2.nome, (parent1.idade + parent2.idade) // 2, parent2.sexo, parent2.peso,parent2.altura)
            new_generation.extend([child1, child2])
        self.individuals.extend(new_generation)

    def mutation(self):
        """ A mutação introduz variações aleatórias nos genomas dos indivíduos. Isso simula as mutações aleatórias no DNA que ocorrem na natureza. O objetivo da mutação em algoritmos evolutivos é manter a diversidade genética na população, prevenindo a convergência prematura para ótimos locais. Alterações pequenas e aleatórias nos genes podem levar a descobertas de novas e melhores soluções."""
        # Simple mutation: randomly change the age
        for individual in self.individuals:
            if random.random() < 0.1:  # 10% chance to mutate
                individual.idade = fake.random_int(min=18, max=80)
                individual.fitness = individual.calculate_fitness()

### Manipulação de dados


In [123]:


class dataManipulation:
    def __init__(self):
        self.df = pd.DataFrame()
        self.array = []

    def gerarDataframe(self, population):
        # Cria uma lista de dicionários, cada um representando um indivíduo
        data = [{
            'nome': individual.nome,
            'idade': individual.idade,
            'sexo': individual.sexo,
            'peso[KG]': individual.peso,
            'altura': individual.altura,
            'fitness': individual.fitness,
            'QI': individual.QI,

            'IMC': individual.imc

        } for individual in population]

        # Converte a lista de dicionários para um DataFrame
        df = pd.DataFrame(data)
        return df

    # Função para criar o grafo de cruzamentos
    def create_graph(self,population):
        G = nx.DiGraph()
        for individual in population:
            G.add_node(individual.nome, sexo=individual.sexo, imc=individual.imc)

        for i in range(0, len(population), 2):
            G.add_edge(population[i].nome, population[i+1].nome)

        return G

    def criarGrafo(self,population):
        G = nx.DiGraph()
        # Atualize o gráfico com informações desta geração
        for i, individual in enumerate(population):
            G.add_node(individual.nome, sexo=individual.sexo, imc=individual.imc)
            if i % 2 == 0 and (i + 1) < len(population):  # Certifique-se de que i+1 não está fora do intervalo
                G.add_edge(population[i].nome, population[i+1].nome)
        return G  # Certifique-se de retornar o gráfico

    def plot_graph(self, G):
        plt.figure(figsize=(12, 8))
        pos = nx.shell_layout(G)  # Posicionamento em camadas para cada geração
        nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, edge_color='gray')
        plt.show()



## Evolution

In [124]:

class EvolutionaryAlgorithm:
    def __init__(self, population_size, generations):
        self.population = Population(population_size)
        self.generations = generations
        self.rankedSolutions = []
        self.solutions = []
        self.new_generation= []
        self.all_generations_df = pd.DataFrame()  # DataFrame para todas as gerações
        self.elite_set_df = pd.DataFrame() 

    def run(self):
        dsa = dataManipulation()
    
        for _ in range(self.generations):
            print(f"\n========== Generation {_+1} Best Solutions ==========")

            self.population.evaluate()
            self.population.selection()
            self.population.crossover()
            self.population.mutation()

            # Cria um DataFrame para a geração atual e armazena no df consolidado
            current_gen_df = dsa.gerarDataframe(self.population.individuals)
            current_gen_df['Generation'] = _ + 1  # Adiciona uma coluna de geração
            self.all_generations_df = pd.concat([self.all_generations_df, current_gen_df], ignore_index=True)
            
            # Cria e atualiza o DataFrame do conjunto elite
            elite_set = self.population.get_elite_set()
            current_elite_df = dsa.gerarDataframe(elite_set)
            current_elite_df['Generation'] = _ + 1  # Adiciona uma coluna de geração
            self.elite_set_df = pd.concat([self.elite_set_df, current_elite_df], ignore_index=True)

            grafico = dsa.criarGrafo(self.population.individuals)
            #dsa.plot_graph(grafico)

        # Exibe o DataFrame consolidado de todas as gerações e o conjunto elite
        print("\nTodas Gerações")
        display(self.all_generations_df)
        #print(self.all_generations_df.describe())

        print("\nConjunto Elite")
        new_gen = self.elite_set_df.drop_duplicates(subset='nome', keep='first')
        display(new_gen)
        try:
            cruzamentoElite(self.elite_set_df)
        except:
            print("Erro ao criar cruzamento")
        #print(self.elite_set_df.describe())

if __name__ == "__main__":
    
    ea = EvolutionaryAlgorithm(population_size=100, generations=10)
    ea.run()



========== Generation 1 Best Solutions ==========

========== Generation 2 Best Solutions ==========

========== Generation 3 Best Solutions ==========

========== Generation 4 Best Solutions ==========

========== Generation 5 Best Solutions ==========

========== Generation 6 Best Solutions ==========

========== Generation 7 Best Solutions ==========

========== Generation 8 Best Solutions ==========

========== Generation 9 Best Solutions ==========

========== Generation 10 Best Solutions ==========

Todas Gerações


,nome,idade,sexo,peso[KG],altura,fitness,QI,IMC,Generation
0,Otávio Viana,45,M,46,1.39,95.6,128,23.81,1
1,Maria Julia Moreira,37,F,50,1.44,93.5,130,24.11,1
2,Maitê Caldeira,58,F,70,1.71,92.8,122,23.94,1
3,Marina Teixeira,50,F,122,2.06,91.7,123,28.75,1
4,Antônio da Conceição,49,M,118,2.09,91.3,114,27.01,1
...,...,...,...,...,...,...,...,...,...
995,Valentina Sales,54,F,74,1.83,72.4,60,22.10,10
996,Maria Julia Moreira,40,F,50,1.44,67.3,41,24.11,10
997,Melissa da Cunha,40,F,93,1.91,65.2,33,25.49,10
998,Valentina Sales,46,F,74,1.83,90.5,119,22.10,10



Conjunto Elite


,nome,idade,sexo,peso[KG],altura,fitness,QI,IMC,Generation
0,Lara Pires,20,F,88,1.92,84.5,124,23.87,1
4,Valentina Sales,65,F,74,1.83,89.5,130,22.10,3


Erro ao criar cruzamento


In [125]:
import pandas as pd
import random
# Função para selecionar um par e verificar se já foram emparelhados
def select_pair(df_male, df_female, paired_names):
    while True:
        male = df_male.sample(1).iloc[0]
        female = df_female.sample(1).iloc[0]
        
        # Cria uma tupla com os nomes, independente da ordem
        pair = tuple(sorted([male['nome'], female['nome']]))
        
        # Verifica se o par já existe
        if pair not in paired_names:
            paired_names.add(pair)
            return male, female
        
# Suponha que df seja o seu DataFrame
# Separe os DataFrames por sexo
def cruzamentoElite(df):
    df_male = df[df['sexo'] == 'M']
    df_female = df[df['sexo'] == 'F']

    # Lista para armazenar os pares que já foram feitos
    paired_names = set()



    # Seleciona um par
    male_individual, female_individual = select_pair(df_male, df_female, paired_names)

    # Agora você tem dois indivíduos para fazer o "cruzamento"
    # Você pode criar um novo indivíduo (herdeiro) baseado nas características deles
    # Por exemplo:
    herdeiro = {
        'nome': 'NomeDoHerdeiro',
        'idade': 0,  # A idade seria 0 ou algum outro critério
        'sexo': random.choice(['M', 'F']),
        # Outras características podem ser uma média dos pais, ou algum outro critério
        'peso[KG]': (male_individual['peso[KG]'] + female_individual['peso[KG]']) / 2,
        'altura': (male_individual['altura'] + female_individual['altura']) / 2,
        # ... e assim por diante
    }

    # Adiciona o herdeiro ao DataFrame (se necessário)
    df = df.append(herdeiro, ignore_index=True)
    display(df)

## Deap

In [126]:
from deap import base, creator, tools, algorithms
import random
from faker import Faker
import pandas as pd

class Individual:
    def __init__(self, nome, idade, sexo,peso,altura):
        self.nome = nome
        self.idade = idade
        self.sexo = sexo
        self.peso = peso
        self.altura = altura
        self.fitness = self.calculate_fitness()
        self.imc = self.calculate_imc()

    def calculate_imc(self):
        return round(self.peso/(self.altura**2),2)

    # Example fitness function
    def calculate_fitness(self):
        # Hypothetical fitness: closer the age to 30, higher the fitness.
        target_age = 30
        return max(1, 30 - abs(self.idade - target_age))

class Population:
    def __init__(self, size):
        self.individuals = [self.create_individual() for _ in range(size)]
        self.conjunto_elite = []

    def create_individual(self):
        nome = fake.name()
        idade = fake.random_int(min=18, max=80)
        sexo = random.choice(["M", "F"])
        peso = fake.random_int(min=45, max=150)
        altura = round(random.uniform(1.20,2.10),2)
        return Individual(nome, idade, sexo,peso,altura)


fake = Faker("pt_BR")

# Define o ambiente de fitness e o indivíduo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Inicializa o toolbox do DEAP
toolbox = base.Toolbox()
toolbox.register("attribute", fake.random_int, min=18, max=80)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Definir a função de fitness
def evaluate(individual):
    # Suponha que queremos maximizar a idade até 30 anos
    target_age = 30
    return max(1, 30 - abs(individual[0] - target_age)),

# Registrar as operações genéticas
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Definir a função principal do algoritmo genético
def run_evolutionary_algorithm():
    # Cria a população inicial
    population = toolbox.population(n=100)
    
    # Número de gerações
    NGEN = 10

    # Probabilidades de cruzamento e mutação
    CXPB, MUTPB = 0.5, 0.2
    
    # Algoritmo evolutivo
    for gen in range(NGEN):
        # Seleciona a próxima geração
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        # Aplica cruzamento e mutação nos descendentes
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            try:
                if random.random() < CXPB:
                    # Ensure cxpoint1 and cxpoint2 are distinct
                    cxpoint1 = random.randint(1, len(child1) - 2)
                    cxpoint2 = random.randint(cxpoint1 + 1, len(child1) - 1)
                    tools.cxTwoPoint(child1, child2, cxpoint1, cxpoint2)
                    del child1.fitness.values
                    del child2.fitness.values
            except:
                print('erro')
        # Avalia os indivíduos com fitness inválidos
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        population[:] = offspring
        
        # Printa informações da geração atual
        fits = [ind.fitness.values[0] for ind in population]
        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print(f"Geração {gen}:  Min {min(fits)}  Max {max(fits)}  Avg {mean}  Std {std}")

    best_ind = tools.selBest(population, 1)[0]
    print(f"Melhor indivíduo: {best_ind}, {best_ind.fitness.values}")
    
if __name__ == "__main__":
    run_evolutionary_algorithm()


erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
Geração 0:  Min 1.0  Max 30.0  Avg 12.78  Std 10.520057034065928
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
Geração 1:  Min 1.0  Max 30.0  Avg 20.92  Std 8.221532703821103
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
Geração 2:  Min 15.0  Max 30.0  Avg 26.88  Std 3.1281943673627697
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
Geração 3:  Min 22.0  Max 30.0  Avg 29.02  Std 1.516443206981407
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
erro
Geração 4:  Min 28.0  Max 30.0  Avg 29.95  Std 0.29580398915513456
erro
erro
erro
err

/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
